In [36]:
import re
from datetime import datetime
from presidio_analyzer import PatternRecognizer
from presidio_analyzer import AnalyzerEngine, RecognizerRegistry

from presidio_anonymizer import AnonymizerEngine


import spacy

In [37]:
from modules import pdf_redactor
nlp = spacy.load("en_core_web_sm")

In [38]:
l1="My phone number is 212-555-5555 From Gerry Kreese "
l2="Sent: Friday 13, September 2019 09:12 To: customerservices@abc.ie "
l3="Subject: 1-541-754-3010 you@email.co.uk yourwebsite.co.uk 194-05-7176 "
l4="Display issue with tv screen Hello ABC, My name is Gerry Kreese and I bought a Sony tv from your "
l5="Philadelphia center last Sunday and I'm facing some display issue with the screen. "
l6="Can you please send someone to fix it. I live just across the 203 Christopher Columbus Blvd, "
l7="Philadelphia and for any other query you can contact me at +12676781822. "
l8="Thanks Gerry Kreese  1-541-754-3010 you@email.co.uk yourwebsite.co.uk 194-05-7176"

text = l1+l2+l3+l4+l5+l6+l7+l8

In [39]:
analyzer = AnalyzerEngine()

In [40]:
results = analyzer.analyze(text=text,
                           entities=["PHONE_NUMBER", "DOMAIN_NAME", "EMAIL_ADDRESS",
                                    "LOCATION", "PERSON", "PHONE_NUMBER"],
                           language='en')
# titles_recognizer = PatternRecognizer(supported_entity="TITLE",
#                                       deny_list=["Mr.","Mrs.","Miss"])

# results = titles_recognizer.analyze(text, entities="TITLE")

In [41]:
print(results)

[type: EMAIL_ADDRESS, start: 92, end: 115, score: 1.0, type: DOMAIN_NAME, start: 109, end: 115, score: 1.0, type: EMAIL_ADDRESS, start: 140, end: 155, score: 1.0, type: DOMAIN_NAME, start: 144, end: 155, score: 1.0, type: DOMAIN_NAME, start: 156, end: 173, score: 1.0, type: EMAIL_ADDRESS, start: 568, end: 583, score: 1.0, type: DOMAIN_NAME, start: 572, end: 583, score: 1.0, type: DOMAIN_NAME, start: 584, end: 601, score: 1.0, type: PHONE_NUMBER, start: 19, end: 31, score: 0.85, type: PERSON, start: 37, end: 49, score: 0.85, type: PERSON, start: 237, end: 249, score: 0.85, type: LOCATION, start: 283, end: 295, score: 0.85, type: LOCATION, start: 459, end: 471, score: 0.85, type: PERSON, start: 539, end: 551, score: 0.85, type: PERSON, start: 568, end: 583, score: 0.85, type: PHONE_NUMBER, start: 127, end: 139, score: 0.5, type: PHONE_NUMBER, start: 555, end: 567, score: 0.5]


In [35]:
registry = RecognizerRegistry()
registry.load_predefined_recognizers()
registry.add_recognizer(titles_recognizer)
analyzer = AnalyzerEngine(registry=registry)
results = analyzer.analyze(text=text, language="en")

print(results)

[type: TITLE, start: 12, end: 15, score: 1.0, type: PERSON, start: 16, end: 21, score: 0.85]


In [42]:
anonymizer = AnonymizerEngine()
anonymized_text = anonymizer.anonymize(text=text,analyzer_results=results)


print(anonymized_text)

My phone number is <PHONE_NUMBER> From <PERSON> Sent: Friday 13, September 2019 09:12 To: <EMAIL_ADDRESS> Subject: 1-<PHONE_NUMBER> <EMAIL_ADDRESS> <DOMAIN_NAME> 194-05-7176 Display issue with tv screen Hello ABC, My name is <PERSON> and I bought a Sony tv from your <LOCATION> center last Sunday and I'm facing some display issue with the screen. Can you please send someone to fix it. I live just across the 203 Christopher Columbus Blvd, <LOCATION> and for any other query you can contact me at +12676781822. Thanks <PERSON>  1-<PHONE_NUMBER> <EMAIL_ADDRESS> <DOMAIN_NAME> 194-05-7176


In [20]:
doc = nlp(text)

In [21]:
pii = list()

In [22]:
for token in doc:
    if token.pos_ == 'PROPN':
        pii.append(token.text)

In [23]:
print(set(pii))

{'Columbus', 'ABC', 'September', 'yourwebsite.co.uk', 'Sunday', 'Gerry', 'Hello', 'Kreese', 'Friday', 'Christopher', '+12676781822', 'Blvd', 'Philadelphia', 'you@email.co.uk', 'Sony', 'Sent'}


In [24]:
mob_regex = "(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"
email_regex = "(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"

In [25]:
for token in doc:
    if token.pos_ == 'PROPN':
        pii.append(token.text)
    elif re.search(mob_regex, str(token)):
        pii.append(token.text)
    elif re.search(email_regex, token.text):
        pii.append(token.text)

In [26]:
print(set(pii))

{'Columbus', 'ABC', 'customerservices@abc.ie', 'September', 'yourwebsite.co.uk', 'Sunday', 'Gerry', 'Hello', 'Kreese', 'Friday', 'Christopher', '+12676781822', 'Blvd', 'Philadelphia', 'you@email.co.uk', 'Sony', 'Sent'}


In [27]:
options = pdf_redactor.RedactorOptions()

options.metadata_filters = {
    # "Title": [lambda value : value.upper()],
    "Producer": [lambda value : "My Name"],
    "CreationDate": [lambda value : datetime.utcnow()],
    "DEFAULT": [lambda value : None],
}

In [28]:
options.xmp_filters = [lambda xml : None]

In [29]:
options.content_filters = [
    (
        re.compile(u"[−–—~‐]"),
        lambda m : "-"
    ),

    (
        re.compile(r"(?<!\d)(?!666|000|9\d{2})([OoIli0-9]{3})([\s-]?)(?!00)([OoIli0-9]{2})\2(?!0{4})([OoIli0-9]{4})(?!\d)"),
        lambda m : "XXX-XX-XXXX"
    ),

    (
        re.compile(r"[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+"),
        lambda m : "redacted@email"
    ),
    
    (
        re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"),
        lambda m : "www.redacted-site.com"
    ),
    
    (
        re.compile(r"comment!"),
        lambda m : "annotation?"
    ),
]

options.link_filters = [
    lambda href, annotation : "https://www.google.com" 
]

In [ ]:
pdf_redactor.redactor(options)